In [1]:
import torch

In [2]:
import sys

In [3]:
from torch.nn import parameter

In [4]:
import torch.nn as nn

In [6]:
import syft as sy

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Users/garima/anaconda3/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.2.so'


In [43]:
import torch.nn.functional as f

In [7]:
# create a local object
hook = sy.TorchHook(torch)

In [9]:
x=torch.tensor([1,2,3,4,6])

In [10]:
y=x+x

In [11]:
print(y)

tensor([ 2,  4,  6,  8, 12])


In [12]:
#create a pretend machine owned by pretend person, Bob
bob=sy.VirtualWorker(hook, id="bob")

In [14]:
#lets create some data and send it to Bob's machine

In [15]:
x= torch.tensor([1,2,3,4,5])
y=torch.tensor([1,1,1,1,1])

In [16]:
#lets send our tensors to Bob
x_ptr=x.send(bob)
y_ptr=y.send(bob)

In [17]:
x_ptr

(Wrapper)>[PointerTensor | me:89299866121 -> bob:27954273041]

In [18]:
#Bob has 2 tensors 

In [19]:
bob._objects

{35587770991: <Plan Plan id:35587770991 owner:bob Tags: #fss_eq_plan_1 built>,
 26497265647: <Plan Plan id:26497265647 owner:bob Tags: #fss_eq_plan_2 built>,
 19836614518: <Plan Plan id:19836614518 owner:bob Tags: #fss_comp_plan_1 built>,
 60968470366: <Plan Plan id:60968470366 owner:bob Tags: #fss_comp_plan_2 built>,
 54778247142: <Plan Plan id:54778247142 owner:bob Tags: #xor_add_1 built>,
 96679135351: <Plan Plan id:96679135351 owner:bob Tags: #xor_add_2 built>,
 27954273041: tensor([1, 2, 3, 4, 5]),
 57071533923: tensor([1, 1, 1, 1, 1])}

In [20]:
z= x_ptr+ x_ptr

In [21]:
z

(Wrapper)>[PointerTensor | me:14014806148 -> bob:74178812553]

In [22]:
bob._objects

{35587770991: <Plan Plan id:35587770991 owner:bob Tags: #fss_eq_plan_1 built>,
 26497265647: <Plan Plan id:26497265647 owner:bob Tags: #fss_eq_plan_2 built>,
 19836614518: <Plan Plan id:19836614518 owner:bob Tags: #fss_comp_plan_1 built>,
 60968470366: <Plan Plan id:60968470366 owner:bob Tags: #fss_comp_plan_2 built>,
 54778247142: <Plan Plan id:54778247142 owner:bob Tags: #xor_add_1 built>,
 96679135351: <Plan Plan id:96679135351 owner:bob Tags: #xor_add_2 built>,
 27954273041: tensor([1, 2, 3, 4, 5]),
 57071533923: tensor([1, 1, 1, 1, 1]),
 74178812553: tensor([ 2,  4,  6,  8, 10])}

In [23]:
x_ptr

(Wrapper)>[PointerTensor | me:89299866121 -> bob:27954273041]

In [24]:
x_ptr.location

<VirtualWorker id:bob #objects:9>

In [25]:
x_ptr.id

89299866121

In [26]:
x_ptr.owner

<VirtualWorker id:me #objects:0>

In [27]:
bob

<VirtualWorker id:bob #objects:9>

In [28]:
bob == x_ptr.location

True

In [30]:
x_ptr.id_at_location

27954273041

In [29]:
# a virtual worker me is also created when, hook = sy.TorchHook(torch) command is run

In [33]:
me=sy.local_worker
me

<VirtualWorker id:me #objects:0>

In [34]:
me == x_ptr.owner

True

In [35]:
# to get a pointer back

In [36]:
x_ptr.get()

tensor([1, 2, 3, 4, 5])

In [37]:
z.get()

tensor([ 2,  4,  6,  8, 10])

In [38]:
bob._objects

{35587770991: <Plan Plan id:35587770991 owner:bob Tags: #fss_eq_plan_1 built>,
 26497265647: <Plan Plan id:26497265647 owner:bob Tags: #fss_eq_plan_2 built>,
 19836614518: <Plan Plan id:19836614518 owner:bob Tags: #fss_comp_plan_1 built>,
 60968470366: <Plan Plan id:60968470366 owner:bob Tags: #fss_comp_plan_2 built>,
 54778247142: <Plan Plan id:54778247142 owner:bob Tags: #xor_add_1 built>,
 96679135351: <Plan Plan id:96679135351 owner:bob Tags: #xor_add_2 built>,
 57071533923: tensor([1, 1, 1, 1, 1])}

In [39]:
# x_ptr and z dont reside any more on bob

In [40]:
me

<VirtualWorker id:me #objects:0>

In [41]:
me._objects

{}

In [42]:
# do some operations on remote tensor objects 

In [45]:
x= torch.tensor([1,2,3,4,5]).send(bob)
y=torch.tensor([1,1,1,1,1]).send(bob)

In [46]:
z=x+y

In [47]:
z

(Wrapper)>[PointerTensor | me:33906492399 -> bob:12476248275]

In [48]:
# command was serialized and sent to bob who performed the computation and send the result back to us

In [49]:
z.get()

tensor([2, 3, 4, 5, 6])

In [50]:
# z.get() recieves the result and give back to us

In [51]:
#can apply torch operations as well

In [52]:
x

(Wrapper)>[PointerTensor | me:18782161443 -> bob:14932716644]

In [53]:
y

(Wrapper)>[PointerTensor | me:13018397983 -> bob:31160672997]

In [54]:
z=torch.add(x,y)

In [55]:
z

(Wrapper)>[PointerTensor | me:52354983997 -> bob:2473132573]

In [56]:
z.get()

tensor([2, 3, 4, 5, 6])

In [57]:
# use backpropagation in torch tensors

In [58]:
x= torch.tensor([1,2,3,4,5.], requires_grad=True).send(bob)
y= torch.tensor([1,1,1,1,1.], requires_grad=True).send(bob)

In [59]:
z=(x+y).sum()

In [60]:
z.backward()

(Wrapper)>[PointerTensor | me:47695406621 -> bob:56587721274]

In [61]:
x=x.get()

In [62]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [63]:
x.grad

tensor([1., 1., 1., 1., 1.])